In [ ]:
# Start writing code here...
!pip install imblearn

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model    import *
from sklearn.metrics         import *
from imblearn.pipeline import Pipeline
from sklearn.preprocessing   import StandardScaler
from sklearn.preprocessing   import OneHotEncoder
from sklearn.preprocessing   import LabelEncoder
from   sklearn.compose            import *
from   sklearn.ensemble           import RandomForestClassifier, ExtraTreesClassifier, IsolationForest
from   sklearn.experimental       import enable_iterative_imputer
from   sklearn.impute             import *
from   sklearn.linear_model       import LogisticRegression, PassiveAggressiveClassifier, RidgeClassifier, SGDClassifier
from   sklearn.metrics            import roc_auc_score # We have not covered it yet in class. The basics - AUC is from 0 to 1 and higher is better.
#from   sklearn.pipeline           import Pipeline
from   sklearn.preprocessing      import *
from   sklearn.tree               import *
from   sklearn.metrics            import balanced_accuracy_score
from   sklearn.feature_selection import *

In [ ]:
from sklearn.ensemble import * 

In [ ]:
rgg_rss_data = pd.read_excel('RGG_RSS_final.xlsx')

In [ ]:
# Removing Outliers

In [ ]:
X=rgg_rss_data[['Commodity', '16 gal', '20 gal', '32 gal', '64 gal', '96 gal','1 yrd', '1.5 yrd', '2 yrd', '3 yrd', '4 yrd',
       '5 yrd', '6 yrd', '#Units',
       'Company', 'Has Key']]
y=rgg_rss_data['Time']

X=X.replace('na',np.nan)
X=X.replace('nan',np.nan)
X=X.replace('.',np.nan)

preprocessing = ColumnTransformer([
                            ('scaler',StandardScaler(),[1,2,3,4,5,6,7,8,9,10,11,12,13]),                       
                            ('categorical', OneHotEncoder(handle_unknown='ignore'), [0,14,15])
                            ])

pipe = Pipeline([
                #('col_selection',FunctionTransformer(categorical_col_transform,validate=False)),
                ('imputation',SimpleImputer(missing_values=np.nan,strategy="most_frequent")),
                ('preprocessing',preprocessing),
                ('Isol',IsolationForest(contamination=0.1))]) 


yhat = pipe.fit_predict(X)

mask = yhat != -1
X, y = X[mask], y[mask]

In [ ]:
data=rgg_rss_data[mask]

In [ ]:
X=data[['Commodity', '16 gal', '20 gal', '32 gal', '64 gal', '96 gal','1 yrd', '1.5 yrd', '2 yrd', '3 yrd', '4 yrd',
       '5 yrd', '6 yrd', '#Units', 'Neighborhood',
       'Company', 'Has Key']]
y=data['Time']

In [ ]:
X=X.replace('na',np.nan)
X=X.replace('nan',np.nan)
X=X.replace('.',np.nan)

In [ ]:
categorical_cols = (X.dtypes == object)
preprocessing = ColumnTransformer([('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
                                   ('continuous', StandardScaler(), -categorical_cols)])  

pipe = Pipeline([
                #('col_selection',FunctionTransformer(categorical_col_transform,validate=False)),
                ('imputation',SimpleImputer(missing_values=np.nan,strategy="most_frequent")),
                ('preprocessing',preprocessing),
                ('XGB',GradientBoostingRegressor())]) 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 30, stop = 200, num = 10)]
# Number of features to consider at every split
max_features = ['log2', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 10)]
# Minimum number of samples required to split a node
min_samples_split = [10,15,20]
# Minimum number of samples required at each leaf node
min_samples_leaf = [10,15,20]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Method of selecting loss on regression
loss=['ls','huber', 'quantile']
# Stopping the tree early
n_iter_no_change=[None,3]
## Learning the rate
learning_rate=np.linspace(0.1,5,num=5)

In [ ]:
random_grid = {'XGB__n_estimators': n_estimators,
               'XGB__max_features': max_features,
               'XGB__max_depth': max_depth,
               'XGB__min_samples_split': min_samples_split,
               'XGB__min_samples_leaf': min_samples_leaf,
               'XGB__loss':loss,
               'XGB__n_iter_no_change':n_iter_no_change,
               'XGB__learning_rate':learning_rate}

In [ ]:
xg_random = RandomizedSearchCV(estimator = pipe, param_distributions = random_grid, n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = -1)
xg_random.fit(X, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/model_selection/_search.py:926: RuntimeWarning: overflow encountered in square
  array_means[:, np.newaxis]) ** 2,


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('imputation',
                                              SimpleImputer(strategy='most_frequent')),
                                             ('preprocessing',
                                              ColumnTransformer(transformers=[('categorical',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               Commodity        True
16 gal          False
20 gal          False
32 gal          False
64 gal          False
96 gal          False
1 yrd           False
1.5 yrd         False
2 yrd           False
3 yrd           False
4 yrd           False
5 yrd           False
6 yrd           False
#Units          False
Nei...
                   param_distributions={'XGB__learning_rate': array([0.1  , 1.325, 2.55 , 3.775, 5.   ]),
                                        'XGB__loss'

In [ ]:
pipe=xg_random.best_estimator_

In [ ]:
def Prediction(X_file):

    X_test=X_file[X.columns]
    time_pred=pipe.predict(X_test)
    X_file['time_pred']=time_pred
    
    return X_file

## Feature Importances

In [ ]:
print(xg_random.best_estimator_.named_steps["XGB"].feature_importances_)
print(xg_random.best_estimator_.named_steps["XGB"].estimators_)

#len(X.columns)

[9.61174976e-03 1.16995485e-02 1.71101800e-02 7.58158900e-03
 1.69278821e-02 7.53086428e-05 5.55632060e-03 4.26999170e-03
 4.04242026e-03 1.08489911e-02 4.19143450e-03 1.65799222e-03
 3.44770398e-04 4.83510616e-04 4.18264209e-03 4.57509725e-03
 2.16752416e-02 3.03354384e-03 3.32494462e-03 1.67731197e-02
 4.96369440e-03 8.25870157e-04 7.72419458e-03 1.05549043e-03
 1.73728163e-03 2.28036272e-03 3.24306746e-03 5.57718433e-03
 1.58121555e-03 1.05645119e-02 4.57972324e-04 1.27156086e-03
 4.23362856e-03 2.34288779e-02 5.21997037e-02 1.62066746e-02
 1.01850738e-03 4.16580314e-02 3.92891366e-02 2.98884275e-01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 2.32833955e-01
 1.00998525e-01]
[[DecisionTreeRegressor(criterion='friedman_mse', max_depth=87,
                      max_features='log2', min_samples_leaf=20,
                      min_samples_split=15,
                      random_state=RandomState(MT19937) at 0x7F36AC328AF0)]
 [D

## Testing our code

In [ ]:
test_file=pd.read_excel('/work/Prepared_RS10_Routes975_998-990.xlsx')

In [ ]:
test_file

,Z1SVC#,Commodity,16 gal,20 gal,32 gal,64 gal,96 gal,1 yrd,1.5 yrd,2 yrd,3 yrd,4 yrd,5 yrd,6 yrd,#Units,Day,Company,Has Key,Neighborhood
0,190,R,0,0,0,0,2,0,0,0,0,0,0,0,2,1,1,True,Marina
1,190,R,0,0,0,0,2,0,0,0,0,0,0,0,2,3,1,True,Marina
2,190,R,0,0,0,0,2,0,0,0,0,0,0,0,2,5,1,True,Marina
3,208,R,0,0,0,0,3,0,0,0,0,0,0,0,3,1,1,True,Marina
4,208,R,0,0,0,0,3,0,0,0,0,0,0,0,3,3,1,True,Marina
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4522,1272616,R,0,0,0,1,0,0,0,0,0,0,0,0,1,4,1,False,Marina
4523,1272624,R,0,0,0,1,0,0,0,0,0,0,0,0,1,2,1,False,Marina
4524,1272707,R,0,0,0,1,0,0,0,0,0,0,0,0,1,3,1,False,Marina
4525,1272715,R,0,0,0,0,1,0,0,0,0,0,0,0,1,2,1,False,Marina


In [ ]:
test_with_pred=Prediction(test_file)

In [ ]:
test_with_pred

,Z1SVC#,Commodity,16 gal,20 gal,32 gal,64 gal,96 gal,1 yrd,1.5 yrd,2 yrd,3 yrd,4 yrd,5 yrd,6 yrd,#Units,Day,Company,Has Key,Neighborhood,time_pred
0,190,R,0,0,0,0,2,0,0,0,0,0,0,0,2,1,1,True,Marina,110.285775
1,190,R,0,0,0,0,2,0,0,0,0,0,0,0,2,3,1,True,Marina,110.285775
2,190,R,0,0,0,0,2,0,0,0,0,0,0,0,2,5,1,True,Marina,110.285775
3,208,R,0,0,0,0,3,0,0,0,0,0,0,0,3,1,1,True,Marina,150.687190
4,208,R,0,0,0,0,3,0,0,0,0,0,0,0,3,3,1,True,Marina,150.687190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4522,1272616,R,0,0,0,1,0,0,0,0,0,0,0,0,1,4,1,False,Marina,49.190000
4523,1272624,R,0,0,0,1,0,0,0,0,0,0,0,0,1,2,1,False,Marina,49.190000
4524,1272707,R,0,0,0,1,0,0,0,0,0,0,0,0,1,3,1,False,Marina,49.190000
4525,1272715,R,0,0,0,0,1,0,0,0,0,0,0,0,1,2,1,False,Marina,80.562840


In [ ]:
test_with_pred.to_excel('Final_Predictions_Routes975_998-990.xlsx')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=938c6ad9-491d-4307-bf8a-c751a244ce4f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>